In [1]:
import cv2
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
import numpy as np
from pygame import mixer
import time

pygame 2.5.2 (SDL 2.28.3, Python 3.8.17)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
#  computer vision and audio for a driver drowsiness detection system.
# Pygame mixer Sound object. handling sound in Python.

In [ ]:
# cv2.CascadeClassifier is used to load Haar cascade classifiers for face and eye detection.
# cv2.data.haarcascades provides the path to the Haar cascade XML files provided by OpenCV.
# haarcascade_frontalface_default.xml and haarcascade_eye.xml are used for face and eye detection, respectively.

In [ ]:
# Haar cascades are a machine learning object detection method used for identifying objects in images or video. 
# Haar cascades are particularly popular for detecting faces and eyes, but they can be trained for various objects.

# Training:

# Haar cascades are trained using positive and negative images.
# Positive images contain the target object (e.g., faces), and negative images do not.
# The training process involves extracting features from these images, such as edges, lines, and textures.
# Haar-like Features:

# Haar-like features are rectangular filters that are applied to an image to capture local intensity patterns.
# These features are simple, rectangular filters that can be efficiently computed.
# Integral Image:

# To speed up feature computation, an integral image is calculated for the input image.
# The integral image allows for the rapid calculation of the sum of pixel values in any rectangular region.
# Adaboost:

# An Adaboost classifier is trained on the Haar-like features.
# Adaboost combines weak classifiers (those that perform slightly better than random) into a strong classifier.
# Cascade Classifier:

# The trained Adaboost classifier is then converted into a cascade of classifiers.
# The cascade structure allows for quickly rejecting negative regions that are unlikely to contain the object.
# Sliding Window:

# The cascade is applied to an image using a sliding window approach.
# At each position and scale, the cascade rapidly rejects regions that likely do not contain the object,
# speeding up the overall detection process.
# Haar cascades are known for their efficiency and speed, making them suitable for real-time applications 
# like face and eye detection in images or video streams. OpenCV, a popular computer vision library, 
# provides pre-trained Haar cascade classifiers for various objects, including faces and eyes

In [2]:
mixer.init()
sound = mixer.Sound('alarm.wav')

face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
eye_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_eye.xml")
model = load_model(r'G:\driver drowsiness detection\models\model.h5')


In [ ]:
# video capture object (cap) using OpenCV.
# 0 as the argument typically refers to the default camera device 

In [3]:
lbl=['Close', 'Open']

path = os.getcwd()
cap = cv2.VideoCapture(0)
font = cv2.FONT_HERSHEY_COMPLEX_SMALL
score = 0

In [4]:
#continuously capture frames from the webcam
while(True):
    ret, frame = cap.read()
    height,width = frame.shape[:2]

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    #     Haar cascades (face_cascade and eye_cascade) are used to detect faces and eyes in the captured frame.

    faces = face_cascade.detectMultiScale(gray,minNeighbors = 3,scaleFactor = 1.1,minSize=(25,25))
    eyes = eye_cascade.detectMultiScale(gray,minNeighbors = 1,scaleFactor = 1.1)

    cv2.rectangle(frame, (0,height-50) , (200,height) , (0,0,0) , thickness=cv2.FILLED )

    for (x,y,w,h) in faces:
        cv2.rectangle(frame, (x,y) , (x+w,y+h) , (255,0,0) , 3 )
        
    #  It extracts the region of interest (ROI) corresponding to the detected eyes, 
    #  resizes it to 80x80 pixels, normalizes pixel values, and reshapes it for input to the neural network.
    
    for (x,y,w,h) in eyes:

        eye = frame[y:y+h,x:x+w]
        #eye = cv2.cvtColor(eye,cv2.COLOR_BGR2GRAY)
        eye = cv2.resize(eye,(80,80))
        eye = eye/255
        eye = eye.reshape(80,80,3)
        eye = np.expand_dims(eye,axis=0)
        prediction = model.predict(eye)
        print(prediction)
        
       #Condition for Close
        if prediction[0][0]>0.30:
            cv2.putText(frame,"Closed",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
            cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
            score=score+1
            # print("Close Eyes")
            
            # eyes are closed for an extended period (controlled by the score variable), an alarm sound is played.
            if(score > 20):
                try:
                    sound.play()
                except:  # isplaying = False
                    pass

        #Condition for Open
        elif prediction[0][1] > 0.90:
            score = score - 1
            if (score < 0):
                score = 0
            cv2.putText(frame,"Open",(10,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
            #print("Open Eyes")
            cv2.putText(frame,'Score:'+str(score),(100,height-20), font, 1,(255,255,255),1,cv2.LINE_AA)
    
    # code displays the processed frame with rectangles drawn around detected faces and eyes.
    cv2.imshow('frame',frame)
    
    # The loop continues until the user presses the 'q' key, 
    # at which point the video capture is released, and the OpenCV windows are closed.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

[[0.91568464 0.08431536]]
[[5.21104e-08 1.00000e+00]]
[[0.66591793 0.3340821 ]]
[[0.00557193 0.99442804]]
[[0.03831054 0.9616894 ]]
[[0.02278386 0.9772161 ]]
[[2.4185024e-06 9.9999762e-01]]
[[2.7376403e-13 1.0000000e+00]]
[[0.7854802  0.21451986]]
[[0.33435825 0.6656418 ]]
[[0.27400178 0.7259983 ]]
[[1.5779184e-04 9.9984217e-01]]
[[1.1522221e-09 1.0000000e+00]]
[[0.6105509  0.38944915]]
[[0.13075444 0.8692455 ]]
[[9.9935585e-01 6.4417109e-04]]
[[0.73156065 0.26843932]]
[[0.03109456 0.96890545]]
[[0.05654669 0.9434534 ]]
[[0.86987805 0.1301219 ]]
[[3.298404e-09 1.000000e+00]]
[[0.93690217 0.06309786]]
[[2.5688507e-06 9.9999738e-01]]
[[0.5388746  0.46112537]]
[[0.855752   0.14424804]]
[[0.00249248 0.99750745]]
[[0.01301277 0.98698723]]
[[0.8504359  0.14956409]]
[[1.099326e-09 1.000000e+00]]
[[0.45360696 0.54639304]]
[[1.0785041e-05 9.9998927e-01]]
[[0.4691631  0.53083694]]
[[1.5468428e-04 9.9984527e-01]]
[[0.51314646 0.48685354]]
[[0.37634552 0.6236545 ]]
[[0.12527508 0.874725  ]]
[[0.43